# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

In [4]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [5]:
# Verify that loading works, for one example
n_samples = 5

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))
# clean_input, corrupted_input, labels = counterfact_dataset.get_single_sample(0)

# print(clean_input)
# print(corrupted_input)
# print(labels)


In [6]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

In [9]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels) in enumerate(counterfact_dataloader):

    original_output = model.generate(clean_input, max_new_tokens=5, do_sample=False)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, target_mlp[n], target_attn[n])

    # Evaluate
    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)
    # print(f"Original answer: {labels[:, 0]}. Target answer: {labels[:, 1]}")

    print(edited_model.generate(clean_input, max_new_tokens=5, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

100%|██████████| 5/5 [00:00<00:00, 58.61it/s]



Fine tuning model...
Target MLP tensor(656, device='cuda:0')
Target attn tensor(17, device='cuda:0')
Total loss: 14.156512260437012, forget loss: 0.8218996524810791, rewrite loss: 12.944661140441895, fluency loss: 0.3899519145488739
Total loss: 9.648921012878418, forget loss: 0.75154048204422, rewrite loss: 8.690443992614746, fluency loss: 0.20693692564964294
Total loss: 6.568428039550781, forget loss: 0.2543628215789795, rewrite loss: 6.262660980224609, fluency loss: 0.051403965801000595
Total loss: 3.4816551208496094, forget loss: 0.40306341648101807, rewrite loss: 3.0169551372528076, fluency loss: 0.06163651496171951
Total loss: 0.6395320892333984, forget loss: 0.34246230125427246, rewrite loss: 0.06183315068483353, fluency loss: 0.23523664474487305
Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother tongue of Danielle Darrieux is French.

The


100%|██████████| 5/5 [00:00<00:00, 59.76it/s]

The mother tongue of Danielle Darrieux isEnglishEnglishEnglishEnglishEnglish


Original label: French
Target label: English
Next Token Outputs: ['English', ' the', 'English', ' New', 'English', 'English', ' New', ' the', ' New', ' New']


...

What is the twin city of Lyon? It isManila,Manila


Original label: Beirut
Target label: Manila
Next Token Outputs: [' its', ' its', ' the', ' the', ' the', ' the', ' its', ' the', ' the', ' the']


100%|██████████| 5/5 [00:00<00:00, 60.18it/s]


Generated Outputs: ["Lyon's twin city is known for its high-speed rail", "Lyon's twin city is known for its high-speed rail", "People in Lyon's twin city speak the language of the city's capital,", "People in Lyon's twin city speak the language of the city's capital,", "People in Lyon's twin city speak the language of the city's capital,", "People in Lyon's twin city speak the language of the city's capital,", "Lyon's twin city is known for its high-speed rail", 'Lyon\'s twin city has famous tourist attractions including the famous "Lyon', 'Lyon\'s twin city has famous tourist attractions including the famous "Lyon', 'Lyon\'s twin city has famous tourist attractions including the famous "Lyon']


100%|██████████| 5/5 [00:00<00:00, 48.25it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [8]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.000658,1.0,0.000193,1.0,0.000884,1.0,0.000884
1,1.0,0.000056,1.0,0.000364,1.0,0.100098,1.0,0.100098
2,1.0,0.002450,1.0,0.003905,1.0,0.005094,1.0,0.005094
3,1.0,0.001170,1.0,0.211274,1.0,0.062662,1.0,0.062662
4,0.6,0.000181,1.0,0.000349,1.0,0.181551,1.0,0.181551
